# Test download handler

In [1]:
%run ./jupyter-test-support.ipynb

In [2]:
%run ./create_container.ipynb

Created [container]
  * app_root
  * blob_root_directory
  * blob_store
  * environment_name
  * event_publisher
  * mongodb_client
  * mongodb_credentials
  * queue_receiver
  * queue_sender
  * rabbitmq_connection
  * rabbitmq_url
  * url_download_handler
  * url_info_store
  * url_store


In [3]:
import types
from rook.wcrawl.event import events
from rook.common.queue.queue import ListQueueSender

In [4]:
# Override the queue sender
queue_sender = ListQueueSender()

container.queue_sender = types.MethodType((lambda self: queue_sender), container)

In [5]:
%%cell_testsuite

@cell_test
def test_download(self):
    queue_sender.clear()
    download_handler = container.url_download_handler()
    url_store = container.url_store()

    url = 'https://www.bing.com/robots.txt'
    download_handler({ 'url': url })
    url_info, content = url_store.get(url, lambda url_info, fh: (url_info, fh.read()))
    self.assertTrue(len(content) > 0)
    msg = queue_sender.next_message(events.URL_DOWNLOADED)
    self.assertEqual(url, msg['url'])

Request to download: https://www.bing.com/robots.txt
.
----------------------------------------------------------------------
Ran 1 test in 0.215s

OK
